In [1]:
import pandas as pd
import pyodbc
import sqlite3
import numpy as np

# iedereen heeft andere servername btw

In [2]:
servername = 'LAPTOP-C1FMPSTV\\SQLEXPRESS01'

# Northwind connectie

In [3]:
northwind_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER='+servername+';DATABASE=NorthWind; Trusted_Connection=yes')
nwCursor = northwind_conn.cursor()
nwCursor

# Adventureworks connectie

In [4]:
adventureworks_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER='+servername+';DATABASE=AdventureWorks2019; Trusted_Connection=yes')
awCursor = adventureworks_conn.cursor()
awCursor

# Aenc connectie

In [5]:
aenc_conn = pyodbc.connect('DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../raw-databases/aenc.accdb;')
aencCursor = aenc_conn.cursor()

table_names = [table_info.table_name for table_info in aencCursor.tables()]

# Print table names
print(table_names)

['MSysAccessStorage', 'MSysACEs', 'MSysComplexColumns', 'MSysIMEXColumns', 'MSysIMEXSpecs', 'MSysNameMap', 'MSysNavPaneGroupCategories', 'MSysNavPaneGroups', 'MSysNavPaneGroupToObjects', 'MSysNavPaneObjectIDs', 'MSysObjects', 'MSysQueries', 'MSysRelationships', 'MSysResources', 'bonus', 'customer', 'department', 'employee', 'product', 'region', 'sales_order', 'sales_order_item', 'state']


In [6]:
product = pd.read_sql_query('SELECT * FROM Production.Product', adventureworks_conn)
ProductInventory = pd.read_sql_query('SELECT * FROM Production.ProductInventory', adventureworks_conn)
ProductSubCategory = pd.read_sql_query('SELECT * FROM Production.ProductSubCategory', adventureworks_conn)
ProductCategory=pd.read_sql_query('SELECT * FROM Production.ProductCategory', adventureworks_conn)

C:\Users\sheng\AppData\Local\Temp\ipykernel_15472\1548848216.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  product = pd.read_sql_query('SELECT * FROM Production.Product', adventureworks_conn)
C:\Users\sheng\AppData\Local\Temp\ipykernel_15472\1548848216.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ProductInventory = pd.read_sql_query('SELECT * FROM Production.ProductInventory', adventureworks_conn)
C:\Users\sheng\AppData\Local\Temp\ipykernel_15472\1548848216.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Pr

In [7]:
category = pd.merge(ProductSubCategory,ProductCategory,on="ProductCategoryID",how='inner')
product_quantity = pd.merge(product,ProductInventory,on="ProductID",how="left")
quantity_groupby = product_quantity.groupby(["ProductID","Name","StandardCost","ListPrice"])['Quantity'].sum().reset_index()
product_sumQuantity = pd.merge(product[["ProductID","ProductSubcategoryID","Color"]],quantity_groupby,on="ProductID",how="left")
product_category = pd.merge(category,product_sumQuantity,on="ProductSubcategoryID",how="right")
result = product_category.loc[:,["ProductID","Name","StandardCost","ListPrice","Quantity",'Name_x','Name_y',"Color"]]
result = result.rename(columns={'Name_x': 'Sub_category'})
result = result.rename(columns={'Name_y': 'Category'})
result = result.rename(columns={'Quantity': 'UnitsInStock'})
aw_product = result
aw_product

,ProductID,Name,StandardCost,ListPrice,UnitsInStock,Sub_category,Category,Color
0,1,Adjustable Race,0.0000,0.00,1085.0,NaN,NaN,None
1,2,Bearing Ball,0.0000,0.00,1109.0,NaN,NaN,None
2,3,BB Ball Bearing,0.0000,0.00,1352.0,NaN,NaN,None
3,4,Headset Ball Bearings,0.0000,0.00,1322.0,NaN,NaN,None
4,316,Blade,0.0000,0.00,1361.0,NaN,NaN,None
...,...,...,...,...,...,...,...,...
499,995,ML Bottom Bracket,44.9506,101.24,816.0,Bottom Brackets,Components,None
500,996,HL Bottom Bracket,53.9416,121.49,970.0,Bottom Brackets,Components,None
501,997,"Road-750 Black, 44",343.6496,539.99,153.0,Road Bikes,Bikes,Black
502,998,"Road-750 Black, 48",343.6496,539.99,155.0,Road Bikes,Bikes,Black


In [8]:
ProductNW = pd.read_sql_query('SELECT * FROM Products', northwind_conn)
categorynw = pd.read_sql_query('SELECT * FROM Categories', northwind_conn)
product_category = pd.merge(ProductNW,categorynw,on="CategoryID",how="left")
nw_product = product_category.loc[:,["ProductID","ProductName","UnitPrice","UnitsInStock","CategoryName"]]
nw_product = nw_product.rename(columns={'ProductName':'Name'})
nw_product = nw_product.rename(columns={'UnitPrice':'ListPrice'})
nw_product = nw_product.rename(columns={'CategoryName':'Category'})
nw_product.UnitsInStock = nw_product.UnitsInStock.astype(float)
nw_product

C:\Users\sheng\AppData\Local\Temp\ipykernel_15472\2431386331.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ProductNW = pd.read_sql_query('SELECT * FROM Products', northwind_conn)
C:\Users\sheng\AppData\Local\Temp\ipykernel_15472\2431386331.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  categorynw = pd.read_sql_query('SELECT * FROM Categories', northwind_conn)


,ProductID,Name,ListPrice,UnitsInStock,Category
0,1,Chai,18.00,39.0,Beverages
1,2,Chang,19.00,17.0,Beverages
2,3,Aniseed Syrup,10.00,13.0,Condiments
3,4,Chef Anton's Cajun Seasoning,22.00,53.0,Condiments
4,5,Chef Anton's Gumbo Mix,21.35,0.0,Condiments
...,...,...,...,...,...
72,73,R?d Kaviar,15.00,101.0,Seafood
73,74,Longlife Tofu,10.00,4.0,Produce
74,75,Rh?nbr?u Klosterbier,7.75,125.0,Beverages
75,76,Lakkalik??ri,18.00,57.0,Beverages


In [9]:
ProductAenc = pd.read_sql_query('SELECT * FROM Product', aenc_conn)
ProductAenc = ProductAenc.rename(columns={'name':'Name'})
ProductAenc = ProductAenc.rename(columns={'id':'ProductID'})
ProductAenc = ProductAenc.rename(columns={'color':'Color'})
ProductAenc = ProductAenc.rename(columns={'quantity':'UnitsInStock'})
ProductAenc = ProductAenc.rename(columns={'unit_price':'ListPrice'})
aencProdcut = ProductAenc.loc[:,["ProductID","Name","ListPrice","UnitsInStock","Category","Color"]]
aencProdcut.UnitsInStock=aencProdcut.UnitsInStock.astype(float)
aencProdcut

C:\Users\sheng\AppData\Local\Temp\ipykernel_15472\2281980832.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ProductAenc = pd.read_sql_query('SELECT * FROM Product', aenc_conn)


,ProductID,Name,ListPrice,UnitsInStock,Category,Color
0,300,Tee Shirt,9,18.0,Clothes,White
1,301,Tee Shirt,14,39.0,Clothes,Orange
2,302,Tee Shirt,14,72.0,Clothes,Black
3,400,Baseball Cap,9,92.0,Accessories,Black
4,401,Baseball Cap,10,12.0,Accessories,White
5,500,Visor,7,36.0,Accessories,White
6,501,Visor,7,28.0,Accessories,Black
7,600,Sweatshirt,24,39.0,Clothes,Green
8,601,Sweatshirt,24,32.0,Clothes,Blue
9,700,Shorts,15,80.0,Clothes,Black


In [10]:
datawarehouseProduct = pd.concat([aw_product,nw_product,aencProdcut],ignore_index=True)
print(datawarehouseProduct)
datawarehouseProduct = datawarehouseProduct.where(pd.notna(datawarehouseProduct), None)
datawarehouseProduct['StandardCost'] = datawarehouseProduct['StandardCost'].fillna(0)
datawarehouseProduct

    ProductID                   Name  StandardCost ListPrice  UnitsInStock  \
0           1        Adjustable Race           0.0       0.0        1085.0   
1           2           Bearing Ball           0.0       0.0        1109.0   
2           3        BB Ball Bearing           0.0       0.0        1352.0   
3           4  Headset Ball Bearings           0.0       0.0        1322.0   
4         316                  Blade           0.0       0.0        1361.0   
..        ...                    ...           ...       ...           ...   
586       500                  Visor           NaN         7          36.0   
587       501                  Visor           NaN         7          28.0   
588       600             Sweatshirt           NaN        24          39.0   
589       601             Sweatshirt           NaN        24          32.0   
590       700                 Shorts           NaN        15          80.0   

    Sub_category     Category  Color  
0            NaN        

,ProductID,Name,StandardCost,ListPrice,UnitsInStock,Sub_category,Category,Color
0,1,Adjustable Race,0.0,0.0,1085.0,None,None,None
1,2,Bearing Ball,0.0,0.0,1109.0,None,None,None
2,3,BB Ball Bearing,0.0,0.0,1352.0,None,None,None
3,4,Headset Ball Bearings,0.0,0.0,1322.0,None,None,None
4,316,Blade,0.0,0.0,1361.0,None,None,None
...,...,...,...,...,...,...,...,...
586,500,Visor,0.0,7,36.0,None,Accessories,White
587,501,Visor,0.0,7,28.0,None,Accessories,Black
588,600,Sweatshirt,0.0,24,39.0,None,Clothes,Green
589,601,Sweatshirt,0.0,24,32.0,None,Clothes,Blue


In [11]:
datawarehouseProduct.UnitsInStock=datawarehouseProduct.UnitsInStock.astype(int)
datawarehouseProduct.ListPrice=datawarehouseProduct.ListPrice.astype(float)
datawarehouseProduct.ProductID=datawarehouseProduct.ProductID.astype(int)
datawarehouseProduct.dtypes

ProductID         int32
Name             object
StandardCost    float64
ListPrice       float64
UnitsInStock      int32
Sub_category     object
Category         object
Color            object
dtype: object

In [13]:
database = "United_outdoors"
export_conn = pyodbc.connect("DRIVER={SQL SERVER};SERVER="+servername + ";DATABASE="+database+";Trusted_Connection=yes")
export_cursor = export_conn.cursor()
export_cursor

In [14]:
for index, row in datawarehouseProduct.iterrows():
    try:
        query = """INSERT INTO product (
            product_productid,
            product_name, 
            product_color, 
            product_standardcost,
            product_listprice,
            product_category,
            product_subcategory,
            product_quantity
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""  
        
        values = (
            row['ProductID'],
            row['Name'],
            row['Color'],
            row['StandardCost'],
            row['ListPrice'],
            row['Category'],
            row['Sub_category'],
            row['UnitsInStock'],

        )

        export_cursor.execute(query, values)

    except pyodbc.IntegrityError as e:
        continue
    except pyodbc.Error as e:
        print("Er is een fout opgetreden: ", e)
        print("Foutieve query: ", query)
    
export_conn.commit()